In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.scale as scale
from matplotlib.animation import FuncAnimation
import numpy as np
import math

base_path = "../build/03_sine_waves"

values = pd.read_csv(f"{base_path}/combined.csv")

values['time'] = values.index * (1/44100)



values[-5000:].plot(x='time', figsize=(30, 5))


In [ ]:
fft = pd.read_csv(f"{base_path}/mags.csv", names=['Frequency Bins'])
fft['freq'] = fft.index * (44100)/(1024 * 1024)
fft.plot(x='freq', y='Frequency Bins', figsize=(10,5))

In [ ]:
# Create a Hann window
x = np.linspace(0, len(values.index), len(values.index))
y = 0.5 * (1 - np.cos(2.0 * np.pi * x / len(values.index)))

plt.plot(x, y)

In [5]:
note_names = [
    'A0', 'B', 
    'C1', 'D', 'E', 'F', 'G', 'A', 'B',
    'C2', 'D', 'E', 'F', 'G', 'A', 'B',
    'C3', 'D', 'E', 'F', 'G', 'A', 'B',
    'C4', 'D', 'E', 'F', 'G', 'A', 'B',
    'C5', 'D', 'E', 'F', 'G', 'A', 'B',
    'C6', 'D', 'E', 'F', 'G', 'A', 'B',
    'C7', 'D', 'E', 'F', 'G', 'A', 'B',
    'C8'
    ]

notes = [
    27.5, 30.868,
    32.7, 36.7, 41.2, 43.7, 49, 55, 61.7,
    65.4, 73.4, 82.4, 87.3, 98, 110, 123.5,
    130.8, 146.8, 164.8, 174.6, 196, 220, 246.9,
    261.6, 293.7, 329.6, 349.2, 392, 440, 493.9,
    523.3, 587.3, 659.3, 698.5, 784, 880, 987.8,
    1046.5, 1174.7, 1318.5, 1396.9, 1568, 1760, 1979.5,
    2093, 2349.3, 2637, 2793.8, 3136, 3520, 3951.1, 
    4186
]

#notes = [16.35, 32.70, 43.7, 65.41, 87.3, 130.81, 174.6, 261.63, 349.2, 523.25, 698.5, 1046.50, 1396.9, 2093.0, 2793.8, 4186.01]
#

In [ ]:
fft = pd.read_csv(f"{base_path}/raw.csv", names=['Frequency Bins'])
fft['db'] = 20 * np.log10(fft['Frequency Bins'])

sample_rate = 48000
n_bins = 8192
step = 1024
frequency_cutoff = 4200
time_per_fft = step / sample_rate

frequencies = np.fft.fftfreq(n_bins, 1 / sample_rate)

hz_per_bin = sample_rate / n_bins
freq_bin_end = int(math.ceil(frequency_cutoff / hz_per_bin))

num_frames = len(fft) // freq_bin_end
fft_data = fft['db'].values.reshape(num_frames, freq_bin_end)

fig, ax = plt.subplots()
fig.set_dpi(1920/16)
fig.set_figheight(9)
fig.set_figwidth(16)
line, = ax.plot(frequencies[:freq_bin_end], np.zeros(freq_bin_end))
#ax.set_xlim(0, frequency_cutoff)  # Limit x-axis to half of Nyquist frequency
ax.set_xscale('log')
ax.set_xticks(notes, note_names)
ax.set_xlim(27, frequency_cutoff)
ax.grid(linestyle=':')

a = ax.get_xgridlines()
for i, label in enumerate(ax.get_xticklabels()):
    if label.get_text().startswith('C'):
        label.set_fontsize(13)  # Increase font size
        label.set_y(label.get_position()[1] - 0.02)
        a[i].set_color('red')
        a[i].set_linewidth(2)


#ax.set_ylim(0, 1)  # Adjust based on expected FFT magnitude
ax.set_ylim(0, fft_data.max())   # Adjust based on expected FFT magnitude
def update(frame):
    # Update the FFT data in the plot
    print(f'Frame: {frame}')
    line.set_ydata(fft_data[frame, :freq_bin_end])
    return line,

ani = FuncAnimation(fig, update, frames=range(num_frames), interval=time_per_fft * 1000, blit=True)
print(f"Saving at {(1/time_per_fft)} fps")
ani.save('animation.mp4', writer='ffmpeg', fps=(1/time_per_fft))